In [122]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [123]:
import cudf as cd
import cuml as ml
import cupy as cp
import numpy as np
import pandas as pd
import sklearn

In [124]:
# Reading the csv file and converting into dataframe
df = cd.read_csv('/kaggle/input/prediction-stock/IXIC.csv')
data = cd.DataFrame(data = df,columns=['Date','Open','High','Low','Close','Adj Close'])
data['target']=df.iloc[:,-1];
data

In [125]:
data.isnull().sum()
type(data)

In [126]:
# Date Feature is not needed so droping the date feature
df = data.drop(['Date'],axis = 1)

In [127]:
df


In [128]:
# Splitting the data in X and Y 
# Findind the sizde of the dataset
df.shape

In [129]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
Y = cp.asarray(Y)
Y=Y.reshape(-1,1)

In [130]:
# Standardization of Data
from cuml.preprocessing import StandardScaler
cuml_scaler = StandardScaler()
X_scaled = cuml_scaler.fit_transform(X)
X_scaled

In [131]:
from cuml.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_scaled,Y,test_size=0.3,random_state = 42)


In [132]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
y_test = Y_test.astype('float64')

In [133]:
from cuml.linear_model import LinearRegression

In [134]:
#Fitting the model and findind the evalution metrics
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'eig')
reg = lr.fit(X_train,Y_train)
print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)
preds = lr.predict(X_test)
print("Predictions:")
print(preds)
print("MSE:")
print(ml.metrics.regression.mean_squared_error(y_test,preds))
print("R2 Score:")
print(ml.metrics.regression.r2_score(y_test,preds))
print("MAE:")
print(ml.metrics.regression.mean_absolute_error(y_test,preds))
